### Import all required libraries

In [ ]:
from music21 import *
import itertools
from itertools import tee
from collections import OrderedDict
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.random import randn
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.ndimage import gaussian_filter
from tqdm import tqdm

### Transform folder of note csvs to one matrix.

In [ ]:
x = 0
a = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

c = list(dict.fromkeys(list(map(' '.join, itertools.chain(
    itertools.product(a, a), itertools.product(a, a))))))


df2 = pd.DataFrame(columns=c, index=c)


for col in df2.columns:
    df2[col].values[:] = 0

totals = []

midi = pd.read_csv('data/midi.csv')
for index, row in midi.iterrows():
    try:
        filepath = 'data/pieces/' + row['File Path'] + '.csv'
        df1 = pd.read_csv(filepath, index_col=0)
        totals.append(df1.to_numpy().sum())
        df2 = pd.concat([df1, df2]).groupby(level=0).sum()
    except:
        print(f'Failed with {filepath}')
df2 = df2.reindex(a)

df4 = df2

for note in a:
    df4.loc[note] = df2.loc[note] / df2.sum(axis=1)[note]
df4

### correctNote()
Function to correct sharps and flats to the same note

In [ ]:
def correctNote(note):
    note_letter = note[:1]

    note_step = list(map(lambda i:i, note[1:]))
    
    note_position = a.index(note_letter)
    
    for x in note_step:
        if x == '-':
            if note_position == 0:
                note_position = 11
            else:
                note_position = note_position - 1
        if x == '#':
            if note_position == 11:
                note_position = 0
            else:
                note_position = note_position + 1
    return a[note_position]

In [ ]:
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [ ]:
def getSong(filepath):
    
    global c_dt
    
    song_dict = { i : 0 for i in c }
    filepath_extended = 'data/midi/' + filepath + '.mid'
    s = converter.parse(filepath_extended)
    
    pair_count = 0


    recurseIter = s.recurse()

    chopin_notes = []
    x = 0
    for i in recurseIter.notes:
        
        if 'Chord' in str(i):
            chord_pitches = i.pitches
            for i in chord_pitches:
                chopin_notes.append(correctNote(i.name))
        else:
            chopin_notes.append(correctNote(i.name))
            

    bigram_list_prob = []
    for v, w in pairwise(chopin_notes):
        bigram_prob = big_data[w][v]
        bigram_list_prob.append(bigram_prob)
        
        
    pair_count = len(bigram_list_prob)
    
    melodic_originality = 1-sum(bigram_list_prob)/pair_count
    

    return song_dict, pair_count, bigram_list_prob, melodic_originality

### analyzePiece()
Calculate the melodic originality for a given piece.

In [ ]:
big_data = df4
def analyzePiece(song):
    song_dict, pair_count, bigram_list_prob, melodic_originality = getSong(song)
    df2 = pd.DataFrame(columns=a, index=a)

    x = 0
    for key,value in song_dict.items():
        if value > 0:
            frequency = value/pair_count
            df.loc[x] = [key] + [frequency, value]
            x = x + 1
            notes = key.split(' ')

            note_1 = notes[0]
            note_2 = notes[1]
            df2.loc[note_1,note_2] = value
            print(f'Note: {key}, Frequency:   {frequency}, Value: {value}')
    dataframe_1 = df2.fillna(0)

    dataframe_2 = dataframe_1.div(pair_count)
    
    
    total_prob = sum(bigram_list_prob)

    test = pair_count

    melodic_originality = 1-(total_prob/test)
    return melodic_originality

Iterate through the piece dataframe to calculate the melodic originality for each peice.

In [ ]:
midi["Melodic Originality"] = ""
for index, row in midi.iterrows():
        file_path = row['File Path']
        melodic_originality = analyzePiece(file_path)
        print(f'[{index+1}] Piece: {file_path}, Originality: {melodic_originality}')
        midi.iat[index, 3] = melodic_originality
        midi.to_csv('data/result.csv', index=False)
midi

In [ ]:
repetoire_df = pd.read_csv('data/result.csv')
pd.set_option('display.max_rows', 50)

df2 = pd.DataFrame(columns=a, index=a)

repetoire_df